In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from corpus_reading import read_dump

chords_tsv = 'data/chord_list.tsv'
notes_tsv = 'data/note_list.tsv'
measures_tsv = 'data/measure_list.tsv'
files_tsv = 'data/file_list.tsv'

chords_df = read_dump(chords_tsv)
notes_df = read_dump(notes_tsv, index_col=[0,1,2])
measures_df = read_dump(measures_tsv)
files_df = read_dump(files_tsv, index_col=0)

# Bugfixes
measures_df.loc[(685, 487), 'next'][0] = 488

In [ ]:
files_df

In [ ]:
chords_df

In [ ]:
measures_df

In [ ]:
notes_df

In [ ]:
import harmonic_inference_data as hid
import importlib
importlib.reload(hid)

train_dataset, valid_dataset, test_dataset = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='811split'
)

In [ ]:
import harmonic_inference_models as him
import harmonic_utils
import importlib
importlib.reload(him)

model = him.MusicScoreModel(len(train_dataset[0]['notes'][0]), len(harmonic_utils.CHORD_TYPES) * 12)

In [ ]:
import model_trainer
import importlib
importlib.reload(model_trainer)

from torch.optim import Adam
from torch.nn import CrossEntropyLoss

optimizer = Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
criterion = CrossEntropyLoss()

trainer = model_trainer.ModelTrainer(model, train_dataset=train_dataset, valid_dataset=valid_dataset,
                                     test_dataset=test_dataset, seed=0, num_epochs=20,
                                     optimizer=optimizer, scheduler=None, criterion=criterion,
                                     log_every=1, log_file=None,
                                     save_every=10, save_dir='checkpoints', save_prefix='checkpoint')

In [ ]:
trainer.train()